In [5]:
import libsbml
import pandas as pd
import numpy as np
from tools import *

In [24]:
model_name = "Selvaggio2020"
xlsx_file = f"Sources/{model_name}.xlsx"
sbml_file = f"Sources/{model_name}.sbml"
save_file = f"Models/{model_name}_annotated.sbml"
strip_file = f"Models/{model_name}_no_annotation.sbml"

knowledge_source = 'pubmed'
qualifier = 'bqbiol:isDescribedBy'

## Add metaids to all components

In [3]:
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_file)
model = document.getModel().getPlugin('qual')

# Ensure each component in the SBML model has a metaid.
def generateMetaId(index):
    return f"metaid_{index:07d}"
    
index = 1
# Process the model
document.getModel().setMetaId(generateMetaId(index))

# Process species
for species in model.getListOfQualitativeSpecies():
    index += 1
    if not species.isSetMetaId():
        species.setMetaId(generateMetaId(index))

# Process transitions
for transition in model.getListOfTransitions():
    index += 1
    if not transition.isSetMetaId():
        transition.setMetaId(generateMetaId(index))
    
    # set metaid for inputs of transition
    for input in transition.getListOfInputs():
        index += 1
        if not input.isSetMetaId():
            input.setMetaId(generateMetaId(index))
    
    # set metaid for outputs of transition
    for output in transition.getListOfOutputs():
        index += 1
        if not output.isSetMetaId():
            output.setMetaId(generateMetaId(index))

## Add notes to transition

In [6]:
# Excel file with the rules and comments
rules_df = pd.read_excel(xlsx_file, sheet_name='Logical_rules')

# Add comments from Excel file to transition notes
for index, row in rules_df.iterrows():
    if row['Comments'] is not np.nan:
        transition_id = f"tr_{row['Node']}"
        transition = model.getTransition(transition_id)
        if transition is not None:
            transition.setNotes(row['Comments'], True)

1

## Add annotations and notes to transition inputs

In [15]:
# Excel file with the transition annotations
transitions_df = pd.read_excel(xlsx_file, sheet_name='Interactions_annotations')

# Go over each row in the transitions_df
for index, row in transitions_df.iterrows():
    # Get the transition id
    transition_id = f"tr_{row['Target']}"
    oneTransition = model.getTransition(transition_id)
    if oneTransition is not None:
        # Get the transition inputs
        oneInput = oneTransition.getListOfInputs().getBySpecies(row['Source'])
        if oneInput is not None:
            meta_id = oneInput.getMetaId()
            list_of_ids = str(row['PMIDs']).split(';')

            # Add annotations
            if len(list_of_ids) > 0:
                AnnotationString = createAnnotationString(qualifier,knowledge_source,RDF_TAG,list_of_ids, meta_id)
                oneInput.setAnnotation(AnnotationString)

            # Add notes
            if row['Comments'] is not np.nan:
                oneInput.setNotes(row['Comments'], True)

1

## Save the model

In [17]:
libsbml.writeSBMLToFile(document, save_file)

1

## Strip the annotations and notes

In [25]:
reader = libsbml.SBMLReader()
document = reader.readSBML(save_file)
model = document.getModel().getPlugin('qual')

# Process species
for species in model.getListOfQualitativeSpecies():
    species.unsetAnnotation()
    species.unsetNotes()

# Process transitions
for transition in model.getListOfTransitions():
    transition.unsetAnnotation()
    transition.unsetNotes()
    for input in transition.getListOfInputs():
        input.unsetAnnotation()
        input.unsetNotes()
    for output in transition.getListOfOutputs():
        output.unsetAnnotation()
        output.unsetNotes()

libsbml.writeSBMLToFile(document, strip_file)

1

## Test

In [16]:
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml_file)
model = document.getModel().getPlugin('qual')
oneTransition = model.getTransition('tr_EGFR')
oneInput = oneTransition.getListOfInputs().getBySpecies('EGF')
meta_id = oneInput.getMetaId()
list_of_ids = ['123456','789000']
AnnotationString = createAnnotationString(qualifier,knowledge_source,list_of_ids, meta_id)
oneInput.setAnnotation(AnnotationString)
oneInput.setNotes('This is a notes test', True)

'<annotation>This is a test</annotation>'

In [26]:
anno = createAnnotationString(qualifier,knowledge_source,['12954170','7680558'],'000000')
print(anno)

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
    <rdf:Description rdf:about="#000000">
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/pubmed/12954170"/>
          <rdf:li rdf:resource="http://identifiers.org/pubmed/7680558"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [22]:
reader = libsbml.SBMLReader()
document = reader.readSBML(save_file)
model = document.getModel().getPlugin('qual')
oneSpecies = model.getQualitativeSpecies('EGF')
print(oneSpecies.getAnnotationString())

<annotation>
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:vCard="http://www.w3.org/2001/vcard-rdf/3.0#" xmlns:vCard4="http://www.w3.org/2006/vcard/ns#" xmlns:bqbiol="http://biomodels.net/biology-qualifiers/" xmlns:bqmodel="http://biomodels.net/model-qualifiers/">
    <rdf:Description rdf:about="#metaid_0000003">
      <bqbiol:is>
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/uniprot/P01133"/>
        </rdf:Bag>
      </bqbiol:is>
    </rdf:Description>
  </rdf:RDF>
</annotation>


In [23]:
oneSpecies.unsetAnnotation()
print(oneSpecies.getAnnotationString())